# Option A - Generate Sphere Manually

In [1]:
out_dir = '/Volumes/OneTouch/datasets/VOSS_STUDIES_TMS/HEBSCHER_TMS/derivatives'
mask_path = '/Users/cu135/hires_backdrops/MNI/MNI152_T1_2mm_brain_mask.nii'

In [3]:
#Enter coordinates of sphere
x = -30
y = -51
z = -9
radius=5

Generate the Sphere at The Coordinate

In [ ]:
import nibabel as nib
import numpy as np
brain_img = nib.load(mask_path)
mni_arr = np.array([x,y,z])

In [ ]:
from calvin_utils.nifti_utils.generate_spherical_roi import calculate_voxelwise_mni_coords, calculate_pairwise_distance, generate_sphere_mask, points_within_sphere, save_mask, mask_within_brain
mni_coords_brain = calculate_voxelwise_mni_coords(brain_img)
distances = calculate_pairwise_distance(mni_arr, mni_coords_brain)
points_in_sphere = points_within_sphere(distances, radius)
sphere_mask = generate_sphere_mask(points_in_sphere, brain_img)
sphere_mask = mask_within_brain(sphere_mask, brain_img)
img = save_mask(sphere_mask, brain_img, mni_arr, out_dir)
img

In [ ]:
import nilearn
nilearn.datasets.fetch_atlas_talairach('ba', data_dir='/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/talairach_spaces', verbose=1)

# Option B - Generate Spheres from a CSV

In [73]:
out_dir = '/Volumes/OneTouch/datasets/VOSS_STUDIES_TMS/HERMILLER_TMS/derivatives'
mask_path = '/Users/cu135/hires_backdrops/MNI/MNI152_T1_2mm_brain_mask.nii'

In [74]:
# Specify the path to your CSV file containing NIFTI paths
input_csv_path = '/Volumes/OneTouch/datasets/VOSS_STUDIES_TMS/HERMILLER_TMS/metadata/master_list.csv'
sheet = None #'master_list_proper_subjects'

In [75]:
from calvin_utils.permutation_analysis_utils.statsmodels_palm import CalvinStatsmodelsPalm
# Instantiate the PalmPrepararation class
cal_palm = CalvinStatsmodelsPalm(input_csv_path=input_csv_path, output_dir=out_dir, sheet=sheet)
# Call the process_nifti_paths method
data_df = cal_palm.read_and_display_data()
data_df

,SUBJECT,stimsite_TLRC,_change_memory_score__post_stim_vs__pre_stim_,Study,Notes
0,CNDR001,52 48 43,0.200000,"Hermiller et al., 2019, Brain & Behav",percent change in cued associative verbal reca...
1,CNDR002,46 66 38,1.000000,NaN,five consecutive days of stimulation (HCN-targ...
2,CNDR004,50 66 35,0.083333,NaN,versus sham stimulation
3,CNDR005,47 65 28,0.363636,NaN,vs sham stimulation
4,CNDR006,42 69 35,0.100000,NaN,NaN
5,CNDR007,44 63 40,1.500000,NaN,NaN
6,CNDR009,47 66 26,0.250000,NaN,NaN
7,CNDR010,44 62 42,0.076923,NaN,NaN
8,CNDR011,45 70 31,0.000000,NaN,NaN
9,CNDR012,47 63 25,0.333333,NaN,NaN



# Preprocess Your Data

**Handle NANs**
- Set drop_nans=True is you would like to remove NaNs from data
- Provide a column name or a list of column names to remove NaNs from

In [76]:
data_df.columns

Index(['SUBJECT', 'stimsite_TLRC',
       '_change_memory_score__post_stim_vs__pre_stim_', 'Study', 'Notes'],
      dtype='object')

In [77]:
drop_list = ['SUBJECT']

In [78]:
data_df = cal_palm.drop_nans_from_columns(columns_to_drop_from=drop_list)

Create x,y,z columns if not present

In [79]:
coord_col = 'stimsite_TLRC'

In [80]:
import numpy as np 
import pandas as pd
try:
    data_df[coord_col] = data_df[coord_col].apply(lambda x: np.array(x.split(), dtype=float))
except:
    print('column already prepared')
data_df[['x_tal', 'y_tal', 'z_tal']] = pd.DataFrame(data_df[coord_col].tolist(), index=data_df.index)
data_df

,SUBJECT,stimsite_TLRC,_change_memory_score__post_stim_vs__pre_stim_,Study,Notes,x_tal,y_tal,z_tal
0,CNDR001,"[52.0, 48.0, 43.0]",0.200000,"Hermiller et al., 2019, Brain & Behav",percent change in cued associative verbal reca...,52.0,48.0,43.0
1,CNDR002,"[46.0, 66.0, 38.0]",1.000000,NaN,five consecutive days of stimulation (HCN-targ...,46.0,66.0,38.0
2,CNDR004,"[50.0, 66.0, 35.0]",0.083333,NaN,versus sham stimulation,50.0,66.0,35.0
3,CNDR005,"[47.0, 65.0, 28.0]",0.363636,NaN,vs sham stimulation,47.0,65.0,28.0
4,CNDR006,"[42.0, 69.0, 35.0]",0.100000,NaN,NaN,42.0,69.0,35.0
5,CNDR007,"[44.0, 63.0, 40.0]",1.500000,NaN,NaN,44.0,63.0,40.0
6,CNDR009,"[47.0, 66.0, 26.0]",0.250000,NaN,NaN,47.0,66.0,26.0
7,CNDR010,"[44.0, 62.0, 42.0]",0.076923,NaN,NaN,44.0,62.0,42.0
8,CNDR011,"[45.0, 70.0, 31.0]",0.000000,NaN,NaN,45.0,70.0,31.0
9,CNDR012,"[47.0, 63.0, 25.0]",0.333333,NaN,NaN,47.0,63.0,25.0


Convert Coordinates to Different Space

In [81]:
# # Perform the conversion
from calvin_utils.nifti_utils.convert_coordinates import convert_coordinates_in_df
df_converted = convert_coordinates_in_df(data_df, 'x_tal', 'y_tal', 'z_tal', 'talairach_to_mni')
display(df_converted)

,SUBJECT,stimsite_TLRC,_change_memory_score__post_stim_vs__pre_stim_,Study,Notes,x_tal,y_tal,z_tal,x_mni,y_mni,z_mni
0,CNDR001,"[52.0, 48.0, 43.0]",0.200000,"Hermiller et al., 2019, Brain & Behav",percent change in cued associative verbal reca...,52.0,48.0,43.0,52.525253,47.203871,49.286739
1,CNDR002,"[46.0, 66.0, 38.0]",1.000000,NaN,five consecutive days of stimulation (HCN-targ...,46.0,66.0,38.0,46.464646,65.994130,44.837191
2,CNDR004,"[50.0, 66.0, 35.0]",0.083333,NaN,versus sham stimulation,50.0,66.0,35.0,50.505051,66.148754,41.580565
3,CNDR005,"[47.0, 65.0, 28.0]",0.363636,NaN,vs sham stimulation,47.0,65.0,28.0,47.474747,65.479956,33.927428
4,CNDR006,"[42.0, 69.0, 35.0]",0.100000,NaN,NaN,42.0,69.0,35.0,42.424242,69.237513,41.743592
5,CNDR007,"[44.0, 63.0, 40.0]",1.500000,NaN,NaN,44.0,63.0,40.0,44.444444,62.802289,46.845248
6,CNDR009,"[47.0, 66.0, 26.0]",0.250000,NaN,NaN,47.0,66.0,26.0,47.474747,66.612624,31.810686
7,CNDR010,"[44.0, 62.0, 42.0]",0.076923,NaN,NaN,44.0,62.0,42.0,44.444444,61.669621,48.961990
8,CNDR011,"[45.0, 70.0, 31.0]",0.000000,NaN,NaN,45.0,70.0,31.0,45.454545,70.473264,37.455766
9,CNDR012,"[47.0, 63.0, 25.0]",0.333333,NaN,NaN,47.0,63.0,25.0,47.474747,63.575407,30.562117


Save changes to CSV if needed

In [82]:
data_df.to_csv(input_csv_path.split('.csv')[0] + '_updated_coords.csv')

Generate spheres given x, y, and z columns

In [83]:
radius = 5
xcol = 'x_mni'
ycol = 'y_mni'
zcol = 'z_mni'
subject_col = 'SUBJECT'

In [84]:
from calvin_utils.nifti_utils.generate_spherical_roi import generate_spherical_rois_from_df
generate_spherical_rois_from_df(data_df, xcol, ycol, zcol, subject_col, radius, mask_path, out_dir, project_on_to_brain=True)

/Users/cu135/.local/lib/python3.10/site-packages/nilearn/image/resampling.py:807: UserWarning: Casting data from int32 to float32
  return resample_img(
/Users/cu135/.local/lib/python3.10/site-packages/nilearn/image/resampling.py:297: UserWarning: Resampling binary images with continuous or linear interpolation. This might lead to unexpected results. You might consider using nearest interpolation instead.
  warnings.warn(
/Users/cu135/.local/lib/python3.10/site-packages/nilearn/image/resampling.py:807: UserWarning: Casting data from int32 to float32
  return resample_img(
/Users/cu135/.local/lib/python3.10/site-packages/nilearn/image/resampling.py:297: UserWarning: Resampling binary images with continuous or linear interpolation. This might lead to unexpected results. You might consider using nearest interpolation instead.
  warnings.warn(
/Users/cu135/.local/lib/python3.10/site-packages/nilearn/image/resampling.py:807: UserWarning: Casting data from int32 to float32
  return resample_

Done
- Calvin